In [4]:
import numpy as np 
from scipy.optimize import root

In [ ]:
def locate_roots(func, x_eval):
    intervals = []
    f = func(x_eval)
    
    for i in range(len(f) - 1):
        if f[i] * f[i+1] < 0:
            intervals.append((x_eval[i], x_eval[i+1]))

        elif abs(f[i]) < 1e-12:  
            intervals.append((x_eval[i], x_eval[i]))  
    
    if len(f) > 0 and abs(f[-1]) < 1e-12:
        intervals.append((x_eval[-1], x_eval[-1]))
    
    return intervals


def newton_method(function, x0, tol=1e-6, max_iter=100):

    def derivative(f, x, h=1e-8):
        return (f(x + h) - f(x - h)) / (2 * h)
    
    x = x0
    
    for i in range(max_iter):
        fx = function(x)
        
        if abs(fx) < tol:
            return x
        
        dfx = derivative(function, x)
        
        if abs(dfx) < 1e-12:
            return None
        
        x_new = x - fx / dfx
        
        if abs(x_new - x) < tol:
            return x_new
        
        x = x_new
    
    return x


def secant_method(function, x0, x1, tol=1e-6, max_iter=100):

    fx0 = function(x0)
    fx1 = function(x1)
    
    for i in range(max_iter):
        if abs(fx1) < tol:
            return x1
        
        if abs(fx1 - fx0) < 1e-12:
            return None
        
        x2 = x1 - fx1 * (x1 - x0) / (fx1 - fx0)
        
        if abs(x2 - x1) < tol:
            return x2
        
        x0, x1 = x1, x2
        fx0, fx1 = fx1, function(x2)
    
    return x1

In [44]:
f1 = lambda x: (1/2)**x + 1 - (x-1)**2
f2 = lambda x: np.arctan(x-1) + 2*x

x_eval = np.linspace(-5, 5, 30)

intervals = locate_roots(f1, x_eval)
for interval in intervals:
    root = newton_method(f1, interval[1])
    print(f'Корень по методу Ньютона: {root}')

    root1 = secant_method(f1, interval[0], interval[1])
    print(f'Корень по методу секущих: {root}')

Корень по методу Ньютона: -0.579270132280304
Корень по методу секущих: -0.579270132280304
Корень по методу Ньютона: 2.109809701305168
Корень по методу секущих: 2.109809701305168


In [1]:
def Newton_system(F, J, x, eps):
    x1, x2 = x
    F_value = F(x1, x2)
    F_norm = np.linalg.norm(F_value, ord=2) 
    iteration_counter = 0

    while abs(F_norm) > eps and iteration_counter < 100:
        delta = np.linalg.solve(J(x1, x2), -np.array(F_value))
        x1 = x1 + delta[0]
        x2 = x2 + delta[1]
        F_value = F(x1, x2)
        F_norm = np.linalg.norm(F_value, ord=2)
        iteration_counter += 1

    if abs(F_norm) > eps:
        iteration_counter = -1

    return [x1, x2], iteration_counter

def Relaxation_method(F, x0, Lambda=None, omega=1, eps=1e-6, max_iterations=10000):
    x = np.array(x0, dtype=float)
    n = len(x)
    
    if Lambda is None:
        Lambda = np.eye(n) * omega
    
    iteration_counter = 0
    
    while iteration_counter < max_iterations:
        x_old = x.copy()
        
        F_value = np.array(F(*x))
        x = x - np.dot(Lambda, F_value)
        
        if np.linalg.norm(x - x_old, ord=2) < eps:
            break
            
        iteration_counter += 1
    
    return x, iteration_counter

In [2]:
import sympy as smp

x1, x2 = smp.symbols('x1 x2')

sys1 = [smp.sin(x1+1) - x2 - 1.2,
        2*x1 + smp.cos(x2) - 2]

sys2 = [smp.cos(x1+2) - x2 - 3/2,
        x1 + smp.cos(x2-2) - 1.2]

J1 = smp.Matrix([[smp.diff(eq, var) for var in (x1, x2)] for eq in sys1])
J2 = smp.Matrix([[smp.diff(eq, var) for var in (x1, x2)] for eq in sys2])

F_np1 = smp.lambdify([x1, x2], sys1, 'numpy')
J_np1 = smp.lambdify([x1, x2], J1, 'numpy')

F_np2 = smp.lambdify([x1, x2], sys2, 'numpy')
J_np2 = smp.lambdify([x1, x2], J2, 'numpy')

In [3]:
import numpy as np
Newton_system(F_np1, J_np1, [0,0], 1e-6)

([np.float64(0.5101501574445773), np.float64(-0.2018384153568168)], 4)

In [5]:
Newton_system(F_np2, J_np2, [0,0], 1e-6)

([np.float64(1.4594901937847922), np.float64(-2.449894721388308)], 9)

In [10]:
Relaxation_method(F_np1, [np.float64(0.710150153), np.float64(-0.401838415356)], omega=1e-2)

(array([ 0.51018298, -0.20192276]), 6699)

In [15]:
Relaxation_method(F_np2, [np.float64(1.4937), np.float64(-2.13)], omega=1e-5)

(array([ 1.5252736 , -2.10488474]), 10000)